In [10]:
from pycorese.api import CoreseAPI

python_to_java_bridge = 'py4j'
corese = CoreseAPI(java_bridge=python_to_java_bridge)
_ = corese.loadCorese()


import os

if 'src' in os.getcwd():
    os.chdir('..')

os.getcwd()

2025-04-17 11:39:19,261 - INFO - Py4J: Loading CORESE...
2025-04-17 11:39:20,220 - INFO - Py4J: CORESE is loaded


'/mnt/c/repos/rdf_permissions_example'

In [11]:
g = corese.loadRDF("data/server/graph_data.rdf")

In [12]:
# Simple first 5 triples from the whole graph
res = corese.sparqlSelect(g, "SELECT * WHERE { ?s ?p ?o } LIMIT 5", return_dataframe=True)
res

,s,p,o
0,_:b0,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,local://security.permissions.net/vocab#managedBy
1,_:b1,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,local://security.permissions.net/vocab#manages...
2,_:b0,http://www.w3.org/1999/02/22-rdf-syntax-ns#rest,_:b1
3,_:b1,http://www.w3.org/1999/02/22-rdf-syntax-ns#rest,http://www.w3.org/1999/02/22-rdf-syntax-ns#nil
4,local://security.permissions.net/entities#1787...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,local://security.permissions.net/vocab#Record


In [13]:
# Since we haven't run ontological reasoning yet, the "manages" relationships are not inferred
query = """
PREFIX : <local://security.permissions.net/vocab#>

SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
  FILTER(?p = :manages || ?p = :managesDepartment) .
}

"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,s,p,o
0,local://security.permissions.net/entities#Jackson,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#APPAREL
1,local://security.permissions.net/entities#John,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#HOBBY
2,local://security.permissions.net/entities#John,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#OFFICE
3,local://security.permissions.net/entities#Steve,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#KITCHEN
4,local://security.permissions.net/entities#Sama...,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#HOME...
5,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#CHRI...
6,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#BIRT...
7,local://security.permissions.net/entities#Anya,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#STORAGE


In [14]:
# Run the rules engine
corese.resetRuleEngine(g)
corese.loadRuleEngine(g, profile='owlrl')
corese.loadRuleEngine(g, profile='owlrl_lite')
corese.loadRuleEngine(g, profile='owlrl_ext')
corese.loadRuleEngine(g, profile='rdfs')

JavaObject id=o18

In [15]:
# Now that we have added our inferences, we can see the results of the manages relationships which have been inferred from both managesDepartment and mangedBy
query = """
PREFIX : <local://security.permissions.net/vocab#>

SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
  FILTER(?p = :manages || ?p = :managesDepartment) .
}

"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,s,p,o
0,local://security.permissions.net/entities#Jackson,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#Sale...
1,local://security.permissions.net/entities#Jackson,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#APPAREL
2,local://security.permissions.net/entities#Jackson,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#Sale...
3,local://security.permissions.net/entities#Jackson,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#Sale...
4,local://security.permissions.net/entities#Jackson,local://security.permissions.net/vocab#manages,local://security.permissions.net/entities#Sale...
...,...,...,...
61,local://security.permissions.net/entities#Steve,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#KITCHEN
62,local://security.permissions.net/entities#Sama...,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#HOME...
63,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#CHRI...
64,local://security.permissions.net/entities#Mary,local://security.permissions.net/vocab#manages...,local://security.permissions.net/entities#BIRT...


In [36]:
# Find all the records that a specific user may have access to.

USER = "SalesRep_0"

query = f"""
prefix data: <local://security.permissions.net/entities#>
prefix : <local://security.permissions.net/vocab#>

select ?record where {{
data:{USER} :hasAccess ?item . 
?item a :Item .
?item :hasRecord ?record .
?record a :Record .
optional {{
    data:{USER} :hasAccess ?country .
    ?country a :Country ; :hasRecord ?record .
}}
optional {{
    data:{USER} a :Manager .
    bind(:Manager as ?mgr) .
}}
filter(bound(?country) || bound(?mgr)) .
}}
"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,record
0,local://security.permissions.net/entities#4566...
1,local://security.permissions.net/entities#4566...
2,local://security.permissions.net/entities#4566...
3,local://security.permissions.net/entities#4566...
4,local://security.permissions.net/entities#4566...
...,...
2110,local://security.permissions.net/entities#4916...
2111,local://security.permissions.net/entities#4554...
2112,local://security.permissions.net/entities#4554...
2113,local://security.permissions.net/entities#1782...


In [38]:
# Show that through inference, managers are also users and their records can be queries


USER = "Mary"

query = f"""
prefix data: <local://security.permissions.net/entities#>
prefix : <local://security.permissions.net/vocab#>

select ?record (bound(?mgr) as ?isManager) where {{
data:{USER} :hasAccess ?item . 
?item a :Item .
?item :hasRecord ?record .
?record a :Record .
optional {{
    data:{USER} :hasAccess ?country .
    ?country a :Country ; :hasRecord ?record .
}}
optional {{
    data:{USER} a :Manager .
    bind(:Manager as ?mgr) .
}}
filter(bound(?country) || bound(?mgr)) .
}}
"""

# Execute the query
res = corese.sparqlSelect(g, query, return_dataframe=True)
res

,record,isManager
0,local://security.permissions.net/entities#4654...,True
1,local://security.permissions.net/entities#4745...,True
2,local://security.permissions.net/entities#4745...,True
3,local://security.permissions.net/entities#4745...,True
4,local://security.permissions.net/entities#4773...,True
...,...,...
1153,local://security.permissions.net/entities#4580...,True
1154,local://security.permissions.net/entities#7475...,True
1155,local://security.permissions.net/entities#4413...,True
1156,local://security.permissions.net/entities#7524...,True
